# Notebook Programmation concurrente

Safwane Benidir

Benjamin de Laverny

https://www.codeflow.site/fr/article/python-gil

https://colab.research.google.com/drive/1M3i6J4QKgEssUZxZZGqrSnPIPJ1lSYF_?usp=sharing&fbclid=IwAR0qYPC_TaRpSgdTw_ZUVumZXn8NeEueB515mIbzU8jeC5Ljvls9MVqU-VE

# Introduction

Dans le cadre de notre projet nous devons réaliser des programmes s'exécutant en parallèle. Nous devons alors gérer les éventuels conflits.

Le projet se fait en Python. Dans un premier temps nous allons étudier les bibilotèques Python permettant de réaliser du parallélisme.

# Bibliothèques Python pour réaliser du parallélisme


## Threading

La bibliothèque <b>threading</b> utilise des threads pour réaliser du parallélisme.

Les threads possèdent un espace mémoire partagé ce qui permet de partager facilement des informations. Cependant il y a un fort risque de conflit lors d'écriture de données. La bibliothèque permet alors l'utilisation de verrou pour protéger les données.

Ce système de verrou s'appuie sur le principe du acquire - release. On indique dans un premier temps que le thread prend le verrou s'il est libre, sinon le thread est bloqué. Quand on finit la section critique on peut libérer le verrou et le thread qui était bloqué (s'il y en a un) peut à son tour prendre le verrou.

### Avantages
On peut facilement partager des données entre threads.

Les threads utilisent moins d'espace mémoire car cet espace est partagé entre les threads

On peut découper un problème en sous-problème.

### Inconvénients
Les threads ne bénéficient pas de la présence de plusieurs coeurs sur un processeur. Il n'y a pas de réelle parallélisme.

Il faut faire très attention lors de l'écriture du code car il faut vérifier les éventuels conflits. La mémoire étant partagé il y a plus de risque qu'avec du multiprocessing.

## Multiprocessing
La bibliothèque <b>multiprocessing</b> utilise des processus pour réaliser du parallélisme.

La mémoire n'est pas partagée, chaque processus ayant son propre espace mémoire.

Il est plus difficile de partager des informations car un processus ne peut pas accéder, en théorie, à l'espace mémoire d'un autre processus. Cependant on pourrait penser à 2 processus s'exécutant en parallèle et essayant d'écrire dans un même fichier. On peut également d'utiliser un pipe, c'est ce que propose la biliothèque multiprocessing. Elle met également à disposition des verrous comme la bibliothèque threading. En effet, une grande partie de l'API de threading se retrouve dans multiprocessing.

### Avantages
En créant différents processus on peut bénéficier d'un parallélisme physiquement réelle sur un processeur a plusieurs coeurs.

Si un processus crash, les autres processus peuvent continuer leur exécution.

### Inconvénients
La création de nouveau processus nécessite plus de mémoire.


# Différence entre processus et thread

<img src="schema_prog_sys.png">

(Schéma du cours de prog_sys)

Dans ce schéma nous voyons à gauche un thread seul qui représente donc un processus avec son espace mémoire (pile + registres). Un processus ne peut pas accéder à la mémoire d'un autre processus.

À droite nous avons plusieurs threads qui se partagent le même espace mémoire, chacun ayant sa propre pile et ses propres registres. Un thread peut accéder à la mémoire des autres threads.


## Exemples de codes : partie thread

Les deux programmes suivants présentent l'utilisation de la bibliothèque threading.

Ces deux programmes proviennent du site openclassroom et ont été légèrement modifié.

(https://openclassrooms.com/fr/courses/235344-apprenez-a-programmer-en-python/2235545-faites-de-la-programmation-parallele-avec-threading)

Le premier programme lance 2 threads. Le premier thread affiche les lettres du mot "canard" et le deuxième thread affiche les lettres du mot "TORTUE". À l'exécution on voit les lettres s'afficher dans le désordre. Ce n'est pas un mot puis un autre qui sont affichés. Les threads sont exécutés en parallèle. C'est l'OS qui va choisir de donner la main à tel ou tel thread.

L'ajout d'une quantité aléatoire à la variable attente permet de mettre en évidence que ce n'est pas le programmeur qui choisit quel thread va s'exécuter à un moment donné. En effet un thread peut afficher une ou plusieurs lettres selon ce que l'OS va lui autoriser comme temps d'exécution.

Si l'on veut qu'un thread affiche le mot en entier sans coupure il faut indiquer quelle partie du code doit s'exécuter sans interruption : c'est une section critique. Pour cela on peut utiliser des verrous. C'est l'exemple du deuxième programme ci-dessous.

La section critique de ce programme correspond à la boucle for car c'est à ce moment que l'on va écrire sur la sortie standard. On indique alors au thread de prendre le verrou avec l'objet Lock et la fonction Lock.acquire(). Les autres threads sont bloqués et ne peuvent pas écrire. Quand on a fini d'écrire un mot on libère le verrou avec la fonction Lock.release().

## Exemples de codes : partie multiprocessus

Les deux programmes suivants présentent l'utilisation de la bibliothèque multiprocessing.

Ces deux programmes proviennent du site medium.com et ont été légèrement modifié.

(https://medium.com/swlh/protect-your-shared-resource-using-multiprocessing-locks-in-python-21fc90ad5af1)

Le premier programme lance 2 processus. Le premier processus incrémente une variable partagée (grâce à l'objet Value). Le deuxième processus décrémente cette même variable. On incrémente autant que l'on décrémente. On s'attend à ce que la variable aie la même valeur au début et à la fin du programme. Mais ce n'est pas le cas, la valeur varie d'une exécution à l'autre. 

Le problème est que pendant la mise à jour de la valeur de la variable par une des 2 fonctions, l'autre fonction va accéder à l'ancienne valeur de cette variable. La première fonction n'a pas encore écrit la nouvelle valeur et la deuxième fonction va faire son calcul à partir de cette ancienne valeur. C'est la deuxième fonction qui va écrire en dernier et qui aura donc ommis le calcul de la première fonction.

In [ ]:
import multiprocessing as mp
import time
import random

def increase(balance):
    for i in range(5000):
        balance.value = balance.value + 1
        
def decrease(balance):
    for i in range(5000):
        balance.value = balance.value - 1
        
def main():
    balance = mp.Value('i',5000)
    p1 = mp.Process(target=increase, args=(balance,))
    p2 = mp.Process(target=decrease, args=(balance,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(balance.value)
        
main()

Si on veut éviter ce problème il faut, par exemple, ajouter un verrou. Comme précédemment on identifie la section critique, ici cela correspond à ces 2 lignes de code :

balance.value = balance.value + 1
balance.value = balance.value - 1

On prend le verrou juste avant et on le rend juste après avec les mêmes fonctions que pour les threads (Lock.acquire et Lock.release).

In [ ]:
import multiprocessing as mp
import time
import random

def increase(balance,lock):
    for i in range(5000):
        # on prend le verrou
        lock.acquire()
        # section critique
        balance.value = balance.value + 1
        # fin de la section critique on rend le verrou
        lock.release()
        
def decrease(balance,lock):
    for i in range(5000):
        # on prend le verrou
        lock.acquire()
        # section critique
        balance.value = balance.value - 1
        # fin de la section critique on rend le verrou
        lock.release()
        
def main():
    balance = mp.Value('i',5000)
    lock = mp.Lock()
    p1 = mp.Process(target=increase, args=(balance,lock))
    p2 = mp.Process(target=decrease, args=(balance,lock))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(balance.value)
        
main()

# Conclusion

Python propose différents outils afin de réaliser du parallélisme tout en s'assurant que la propriété de sûreté soit vérifiée. Il nous reste à étudier le sujet précisément pour décider quelle bibliothèque sera la plus adaptée.

imports

In [ ]:
import multiprocessing as mp
import threading as th
import time
import os
import sys

In [27]:
N=10000000

def createProcess(n,fonction=None,args=()):
    p=[]
    a=time.time()
    for i in range(n):
        p.append(mp.Process(target=fonction,args=args))
        p[i].start()
    for i in range(n):
        p[i].join()
    print()
    print("temps global = ",time.time()-a)
    

def createThread(n,fonction=None,args=()):
    t=[]
    a=time.time()
    for i in range(n):
        t.append(th.Thread(target=fonction,args=args))
        t[i].start()
    for i in range(n):
        t[i].join()
    print()
    print("temps global = ", time.time()-a)
    
    
def entrelacement(mot):
    for lettre in mot:
        sys.stdout.write(lettre)
        sys.stdout.flush()
        
        
def entrelacement_safe(mot,lock):
    lock.acquire()
    for lettre in mot:
        sys.stdout.write(lettre)
        sys.stdout.flush()
    lock.release()
        

def core(n,lock):
    a=time.time()
    for i in range(n):
        pass
    lock.acquire()
    print(time.time()-a)
    lock.release()
    
    
def temoin_core(n,args):
    a=time.time()
    for i in range(n):
        core(args)
    print("temps global = ",time.time()-a)
    

def increase(balance):
    for i in range(5000):
        balance = balance + 1
        
def decrease(balance):
    for i in range(5000):
        balance = balance - 1
        
def increase_safe(balance,lock):
    for i in range(5000):
        # on prend le verrou
        lock.acquire()
        # section critique
        balance = balance + 1
        # fin de la section critique on rend le verrou
        lock.release()
        
def decrease_safe(balance,lock):
    for i in range(5000):
        # on prend le verrou
        lock.acquire()
        # section critique
        balance = balance - 1
        # fin de la section critique on rend le verrou
        lock.release()
        
def pid(lock):
    lock.acquire()
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    lock.release()

In [ ]:
createProcess(5)

In [ ]:
createThread(5)

In [22]:
createProcess(3,pid,(lock,))

parent process: 15849
process id: 16511
parent process: 15849
process id: 16514
parent process: 15849
process id: 16521

temps global =  0.06296777725219727


In [23]:
createThread(3,pid,(lock,))

parent process: 2320
process id: 15849
parent process: 2320
process id: 15849
parent process: 2320
process id: 15849

temps global =  0.003160715103149414


In [ ]:
createProcess(5,entrelacement,("bonjour ",)) # problème

In [ ]:
createThread(10,entrelacement,("bonjour ",))

In [ ]:
lock=mp.Lock()
createProcess(5,entrelacement_safe,("bonjour ",lock,)) # pas de problème

In [ ]:
lock=th.Lock()
createThread(10,entrelacement_safe,("bonjour ",lock))

In [ ]:
createProcess(10,core,(N,lock))

In [ ]:
createThread(10,core,(N,lock))

In [26]:
balance=5000
createProcess(1,increase,(balance,))
createProcess(1,decrease,(balance,))

Process Process-51:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-63ad82c617ab>", line 59, in increase
    balance.value = balance.value + 1
AttributeError: 'int' object has no attribute 'value'



temps global =  0.020959854125976562


Process Process-52:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-63ad82c617ab>", line 63, in decrease
    balance.value = balance.value - 1
AttributeError: 'int' object has no attribute 'value'



temps global =  0.023340463638305664
